# Gemma and QWEN7B

This notebook uses two Models, Qwen7B and Gemma, to classify the posts.

It will then ensemble the two predictions. I don't know how to best ensemble the two models yet, for now we'll average.


Credits: Kudos to the authors of these notebooks. Thanks for sharing:
- https://www.kaggle.com/code/yangyefd/batch-gemma3-sample-rules-classification
- https://www.kaggle.com/code/aerdem4/jigsaw-acrc-qwen7b-finetune-logits-processor-zoo

## How to install dependencies

Interactive notebooks: Click on Add-Ons->Install Dependencies, then Run.

Submission: Dependencies will be installed automatically

## Prepare Qwen7b and Gemma

In [7]:
import vllm, torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


In [33]:


qwen7b_model_path = "/kaggle/input/jigsaw-acrc-qwen7b-v01"

qwen7b = vllm.LLM(
    qwen7b_model_path,
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.95,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=4096,
    disable_log_stats=True,
    enable_prefix_caching=True
)
qwen7b_tokenizer = qwen7b.get_tokenizer()

INFO 07-25 10:20:45 [config.py:1604] Using max model len 4096
WARNING 07-25 10:20:45 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 07-25 10:20:46 [cuda.py:103] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-25 10:20:46 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='/kaggle/input/jigsaw-acrc-qwen7b-v01', speculative_config=None, tokenizer='/kaggle/input/jigsaw-acrc-qwen7b-v01', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='

2025-07-25 10:20:51.185111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753438851.205554     195 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753438851.211795     195 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 07-25 10:20:56 [__init__.py:235] Automatically detected platform cuda.
(VllmWorkerProcess pid=195) INFO 07-25 10:20:56 [multiproc_worker_utils.py:226] Worker ready; awaiting tasks
(VllmWorkerProcess pid=195) INFO 07-25 10:20:57 [cuda.py:346] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
(VllmWorkerProcess pid=195) INFO 07-25 10:20:57 [cuda.py:395] Using XFormers backend.


[W725 10:21:08.538054594 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:21:09.870660074 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:21:18.548720317 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 07-25 10:21:28 [__init__.py:1375] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=195) INFO 07-25 10:21:28 [__init__.py:1375] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=195) INFO 07-25 10:21:28 [pynccl.py:70] vLLM is using nccl==2.26.2
INFO 07-25 10:21:28 [pynccl.py:70] vLLM is using nccl==2.26.2


[W725 10:21:28.559105246 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 07-25 10:21:29 [custom_all_reduce_utils.py:208] generating GPU P2P access cache in /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 07-25 10:21:52 [custom_all_reduce_utils.py:246] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=195) INFO 07-25 10:21:52 [custom_all_reduce_utils.py:246] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=195) WARNING 07-25 10:21:52 [custom_all_reduce.py:147] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
WARNING 07-25 10:21:52 [custom_all_reduce.py:147] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
INFO 07-25 10:21:52 [shm_broadcast.py:289] vLLM message queue communication handle

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-25 10:23:09 [default_loader.py:262] Loading weights took 75.93 seconds
(VllmWorkerProcess pid=195) INFO 07-25 10:23:09 [default_loader.py:262] Loading weights took 75.72 seconds
(VllmWorkerProcess pid=195) INFO 07-25 10:23:10 [model_runner.py:1115] Model loading took 7.1217 GiB and 75.897206 seconds
INFO 07-25 10:23:10 [model_runner.py:1115] Model loading took 7.1217 GiB and 76.119988 seconds
(VllmWorkerProcess pid=195) INFO 07-25 10:23:16 [worker.py:295] Memory profiling takes 5.88 seconds
(VllmWorkerProcess pid=195) INFO 07-25 10:23:16 [worker.py:295] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
(VllmWorkerProcess pid=195) INFO 07-25 10:23:16 [worker.py:295] model weights take 7.12GiB; non_torch_memory takes 0.11GiB; PyTorch activation peak memory takes 0.36GiB; the rest of the memory reserved for KV Cache is 6.42GiB.
INFO 07-25 10:23:16 [worker.py:295] Memory profiling takes 6.03 seconds
INFO 07-25 10:23:16 [worker.

In [38]:
!ls /kaggle/input/gemma-7b-gguf

gemma-2-9b-it.Q5_K_M.gguf  gemma-7b-it.Q5_K_M.gguf


In [41]:
import vllm, torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


GEMMA_PATH = "/kaggle/input/gemma-7b-gguf/gemma-7b-it.Q5_K_M.gguf"

gemma = vllm.LLM(
    GEMMA_PATH,
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.95,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=4096,
    disable_log_stats=True,
    enable_prefix_caching=True
)
gemma_tokenizer = gemma.get_tokenizer()

ValidationError: 1 validation error for ModelConfig
  Value error, GGUF model with architecture gemma is not supported yet. [type=value_error, input_value=ArgsKwargs((), {'model': ...attention_dtype': None}), input_type=ArgsKwargs]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Prepare Training data and Examples 

In [35]:
import os, sys
import pandas as pd

data_path = "/kaggle/input/jigsaw-agile-community-rules/test.csv" \
                if os.getenv('KAGGLE_IS_COMPETITION_RERUN') \
                else "/kaggle/input/jigsaw-agile-community-rules/train.csv"

df = pd.read_csv(data_path)
df.head(10)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1
5,5,I live in the US it's it possible to get in tr...,No legal advice: Do not offer or request legal...,soccerstreams,Public School? You are under no legal obligati...,"OP, you need to get the fuck away from your bo...",It looks like it could be a sterile cotton swa...,That is called battery. Two wrongs don't make...,0
6,6,young kitty watching for your buddy get her he...,"No Advertising: Spam, referral links, unsolici...",sex,Save on Medicine!! Save over $700 a month on ...,Make your life comfortable. Get up to 15% Disc...,They have nothing on the platypus though goo.g...,Try and see if someone at www.siddhantayoga.co...,0
7,7,[liệt dương](http://namkhoathientam.com/nguyen...,"No Advertising: Spam, referral links, unsolici...",gifs,EARN MONEY in online . Just Sign up and View f...,You can use www.easy-lol.com/probuilds/\n\nIt ...,HD | [English Stream](http://www.ufc187livestr...,* **SD - http://livestreamnba.ru/2016/12/19/pr...,0
8,8,"Maybe true, but that's very short-sighted. The...",No legal advice: Do not offer or request legal...,The_Donald,"OP, you need to get the fuck away from your bo...",Steal the dogs back and put a lean on all her ...,Is this 100% legal tho? Are their any copyrigh...,"If you masturbate before the age of 18, you're...",1
9,9,you can sue them for negligence and try and re...,No legal advice: Do not offer or request legal...,legaladvice,"IIRC the laws require photo id, and social sec...",Tell them you want to take possession of the a...,That is called battery. Two wrongs don't make...,"Heard you might have their address, it could b...",1


## Preparing Prompts

In [31]:
SYS_PROMPT = """
You are given a comment on reddit. Your task is to classify if it violates the given rule. Only respond Yes/No.
"""

qwen7b_prompts = []
gemma_prompts = []
for i, row in df.iterrows():
    text = f"""
r/{row.subreddit}
Rule: {row.rule}

1) {row.positive_example_1}
Violation: Yes

2) {row.negative_example_1}
Violation: No

3) {row.negative_example_2}
Violation: No

4) {row.positive_example_2}
Violation: Yes

5) {row.body}
"""
    print(text)
    
    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text}
    ]

    qwen_prompt = qwen7b_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    ) + "Violation: "
    qwen7b_prompts.append(qwen_prompt)
    gemma_prompt = gemma_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    ) + "Violation: "
    qwen7b_prompts.append(gemma_prompt)




r/Futurology
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.

1) If you could tell your younger self something different about sex, what would that be?

i AM IN A CONTEST TO WIN FUNDING FOR MY SEX POSITIVE FILM: VOTE HERE:

http://sheknows.offerpop.com/campaign/813112/entry/v144417
Violation: Yes

2) Watch Golden Globe Awards 2017 Live Online in HD Coverage without ADS (VIP STREAMS)
=

HD STREAM QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)
=

HD BROADCASTING QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)
=

Mobile Compatibility: YES
=

NO ADS | NO ADS | ADS
=

Violation: No

3) DOUBLE CEE x BANDS EPPS - "BIRDS"

DOWNLOAD/STREAM:

http://music.theblacksmithed.com/download/birds/
Violation: No

4) hunt for lady for jack off in neighbourhood http://url.inmusi.com/gakq
Violation: Yes

5) Banks don't want you to know this! Click here to know more!



NameError: name 'qwen7b_tokenizer' is not defined

## Predict / Apply MultiChoice processor

In [ ]:
qwen_mclp = MultipleChoiceLogitsProcessor(qwen7b_tokenizer, 
                                     choices=["Yes", "No"])

qwen7b_outputs = qwen7b_llm.generate(
    prompts,
    vllm.SamplingParams(
        seed=0,
        skip_special_tokens=True,
        max_tokens=1,
        logits_processors=[mclp],
        logprobs=len(mclp.choices)

    ),
    use_tqdm=True
)


qwen7b_logprobs = []
for lps in [output.outputs[0].logprobs[0].values() for output in qwen7b_outputs]:
    qwen7b_logprobs.append({lp.decoded_token: lp.logprob for lp in list(lps)})

In [ ]:
gemma_mclp = MultipleChoiceLogitsProcessor(gemma_tokenizer, 
                                     choices=["Yes", "No"])

gemma_outputs = gemma_llm.generate(
    prompts,
    vllm.SamplingParams(
        seed=0,
        skip_special_tokens=True,
        max_tokens=1,
        logits_processors=[mclp],
        logprobs=len(mclp.choices)

    ),
    use_tqdm=True
)


gemma_logprobs = []
for lps in [output.outputs[0].logprobs[0].values() for output in gemma_outputs]:
    gemma_logprobs.append({lp.decoded_token: lp.logprob for lp in list(lps)})

## Averaging predictions and creating a submission file


In [ ]:
# Combine logprobs from Qwen and Gemma using logprob averaging
final_preds = []
for q_lp, g_lp in zip(qwen7b_logprobs, gemma_logprobs):
    avg_yes = (q_lp.get("Yes", -999) + g_lp.get("Yes", -999)) / 2
    avg_no = (q_lp.get("No", -999) + g_lp.get("No", -999)) / 2
    final_preds.append("Yes" if avg_yes > avg_no else "No")

# Format for Kaggle submission
submission_df = pd.DataFrame({
    "example_id": df["example_id"],
    "label": final_preds
})

submission_df.to_csv("submission.csv", index=False)
print("Saved submission.csv")


In [ ]:
import matplotlib.pyplot as plt

# Extract individual logprobs for plotting
qwen_yes = [lp.get("Yes", -999) for lp in qwen7b_logprobs]
qwen_no  = [lp.get("No", -999) for lp in qwen7b_logprobs]

gemma_yes = [lp.get("Yes", -999) for lp in gemma_logprobs]
gemma_no  = [lp.get("No", -999) for lp in gemma_logprobs]

avg_yes = [(qy + gy) / 2 for qy, gy in zip(qwen_yes, gemma_yes)]
avg_no  = [(qn + gn) / 2 for qn, gn in zip(qwen_no, gemma_no)]

# Plot histograms
plt.figure(figsize=(12, 6))
plt.hist(qwen_yes, bins=30, alpha=0.5, label='Qwen7B - Yes', density=True)
plt.hist(qwen_no, bins=30, alpha=0.5, label='Qwen7B - No', density=True)
plt.hist(gemma_yes, bins=30, alpha=0.5, label='Gemma - Yes', density=True)
plt.hist(gemma_no, bins=30, alpha=0.5, label='Gemma - No', density=True)
plt.hist(avg_yes, bins=30, alpha=0.6, label='Avg - Yes', histtype='step', linewidth=2)
plt.hist(avg_no, bins=30, alpha=0.6, label='Avg - No', histtype='step', linewidth=2)

plt.title("Histogram of Logprobs for 'Yes' and 'No' Predictions")
plt.xlabel("Log Probability")
plt.ylabel("Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
